In [1]:
import pandas as pd
import mlflow

In [2]:
mlflow.set_tracking_uri("http://ec2-184-73-140-65.compute-1.amazonaws.com:5000")

In [3]:
client = mlflow.tracking.MlflowClient()

In [4]:
experiments = list(client.search_experiments())
experiment = list(filter(lambda k : k.name=='experiments_rent_model', experiments))[0]
experiment

<Experiment: artifact_location='mlflow-artifacts:/334592644149538691', creation_time=1768879142728, experiment_id='334592644149538691', last_update_time=1768879142728, lifecycle_stage='active', name='experiments_rent_model', tags={}>

In [5]:
experiment_id = experiment.experiment_id

In [6]:
experiment_id

'334592644149538691'

In [7]:
from mlflow.entities import ViewType
runs = client.search_runs(
    experiment_ids = experiment_id,
    filter_string = "",
    max_results=3,
    run_view_type=ViewType.ACTIVE_ONLY,
    order_by=["metrics.MAE ASC"]
)

In [8]:
run = runs[0]

In [9]:
run_id = run.data.tags['Run ID']

In [10]:
run_id

'1444e13bff234a0d95043cb17adf0ea1'

In [11]:
exclude = ['Posted On', 'Area Locality', 'Floor', 'Rent']
data = pd.read_csv('../data/raw.csv')

In [12]:
logged_model = f'runs:/{run_id}/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(data.drop(columns=exclude))

array([12234.61346154,  9085.86538462, 10897.88461538, ...,
       45169.80769231, 36821.91346154, 12824.35769231])

In [14]:
# REGISTRADNDO UN MODELO

In [17]:
mlflow.register_model(f'runs:/{run_id}/model', 'rent_model')

Registered model 'rent_model' already exists. Creating a new version of this model...
2026/01/21 19:54:52 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: rent_model, version 2
Created version '2' of model 'rent_model'.


<ModelVersion: aliases=[], creation_timestamp=1769043292749, current_stage='None', description='', last_updated_timestamp=1769043292749, name='rent_model', run_id='1444e13bff234a0d95043cb17adf0ea1', run_link='', source='mlflow-artifacts:/334592644149538691/1444e13bff234a0d95043cb17adf0ea1/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [18]:
client.transition_model_version_stage(
    name="rent_model",
    version=2,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1769043292749, current_stage='Production', description='', last_updated_timestamp=1769043334698, name='rent_model', run_id='1444e13bff234a0d95043cb17adf0ea1', run_link='', source='mlflow-artifacts:/334592644149538691/1444e13bff234a0d95043cb17adf0ea1/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [19]:
logged_model = f'models:/rent_model/2'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(data.drop(columns=exclude))

array([12234.61346154,  9085.86538462, 10897.88461538, ...,
       45169.80769231, 36821.91346154, 12824.35769231])

In [20]:
logged_model = f'models:/rent_model/Production'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(data.drop(columns=exclude))

array([12234.61346154,  9085.86538462, 10897.88461538, ...,
       45169.80769231, 36821.91346154, 12824.35769231])

In [22]:
data.columns

Index(['Posted On', 'BHK', 'Rent', 'Size', 'Floor', 'Area Type',
       'Area Locality', 'City', 'Furnishing Status', 'Tenant Preferred',
       'Bathroom', 'Point of Contact'],
      dtype='object')

In [23]:
data

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner
...,...,...,...,...,...,...,...,...,...,...,...,...
4741,2022-05-18,2,15000,1000,3 out of 5,Carpet Area,Bandam Kommu,Hyderabad,Semi-Furnished,Bachelors/Family,2,Contact Owner
4742,2022-05-15,3,29000,2000,1 out of 4,Super Area,"Manikonda, Hyderabad",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Owner
4743,2022-07-10,3,35000,1750,3 out of 5,Carpet Area,"Himayath Nagar, NH 7",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Agent
4744,2022-07-06,3,45000,1500,23 out of 34,Carpet Area,Gachibowli,Hyderabad,Semi-Furnished,Family,2,Contact Agent
